# Imports

In [1]:
use_kaggle=False
use_jupyter=True

In [2]:
if use_jupyter:
    print("Using Jupyter")
    !dir "C:/develop/python/Final Project/Data/"
elif use_kaggle:
    print("Using Kaggle")
    !ls "/kaggle/"
else:
    print("Using Colab with drive")
    from google.colab import drive
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/Colab Notebooks/ML course/"

Using Jupyter
 Volume in drive C has no label.
 Volume Serial Number is B0A9-3BF3

 Directory of C:\develop\python\Final Project\Data

09/04/2020  13:50    <DIR>          .
09/04/2020  13:50    <DIR>          ..
09/04/2020  11:29         9,955,043 Data.zip
07/04/2020  16:24    <DIR>          extracted-masks-images
07/04/2020  16:24         1,856,537 extracted_cells.csv
07/04/2020  15:43    <DIR>          masks-images
07/04/2020  15:43    <DIR>          resized-images
               2 File(s)     11,811,580 bytes
               5 Dir(s)  446,589,595,648 bytes free


In [3]:
import numpy as np
import pandas as pd
import uuid
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import cv2
from skimage import io
from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter, ellipse

# Consts and Setups

In [4]:
if use_jupyter:
    root_input_dir = 'C:/develop/python/Final Project/Data/'
    root_output_dir = 'C:/develop/python/Final Project/Data/'
elif use_kaggle:
    root_input_dir = '/kaggle/input/'
    root_output_dir = '/kaggle/working/output/'
else:
    root_input_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'
    root_output_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'

In [5]:
cell_masks_subdirectory = 'extracted-masks-images'
minimum_cover_percent = 89
max_factor_size = 4
uuid_row_index = 1
input_csv_filename = 'extracted_cells.csv'
output_csv_filename = 'filtered_cells.csv'
images_extension = ".png"
csv_field_names = ['source_file', 'mask_uuid', 'left_x', 'right_x', 'top_y', 'bottom_y', 'id']
start_line=10001
end_line=15000

In [6]:
plt.gray()
np.seterr(divide='ignore', invalid='ignore')

# Inputs
input_csv_file_path = root_input_dir + input_csv_filename
masks_images_path = root_input_dir + cell_masks_subdirectory + '/'
input_csv_rows_list = []

# Outputs
output_csv_rows_list = []
output_csv_file_path = root_output_dir + f'{start_line}-{end_line}_' + output_csv_filename

<Figure size 432x288 with 0 Axes>

# Function Definition

In [7]:
def getMaskBestEllipse(mask_data):
    edges = canny(mask_data, sigma=2.0, low_threshold=0.55, high_threshold=0.8)
    ellipses_data = hough_ellipse(edges, accuracy=10, threshold=40, min_size=0, max_size=500)
    ellipses_data.sort(order='accumulator')
    best = list(ellipses_data[-1])
    yc, xc, a, b = [int(round(x)) for x in best[1:5]]
    orientation = best[5]
    return yc, xc, a, b, orientation

In [8]:
def resizeImage(image_data, scale_size):
    width = int(current_mask_data.shape[1] * scale_size)
    height = int(current_mask_data.shape[0] * scale_size)
    dim = (width, height)
    # resize image
    return cv2.resize(current_mask_data, dim, interpolation = cv2.INTER_AREA)

In [9]:
def getEllipseCoverPercent(mask_data, factor_size):
    # Resize the image first
    resized_image = resizeImage(mask_data, factor_size)
    mask_copy = resized_image.copy()

    # Get the best ellipse that covers the mask
    yc, xc, a, b, orientation = getMaskBestEllipse(resized_image)

    # Try to draw the best ellipse on the mask
    try:
        # Get the indices of the ellipse fill and the ellipse perimiter
        perimiter_y, perimiter_x = ellipse_perimeter(yc, xc, a, b, orientation)
        fill_y, fill_x = ellipse(yc, xc, a, b, rotation=-orientation)

        # Set 0 (black) in the mask copy where the indexes are located
        mask_copy[list(perimiter_y), list(perimiter_x)] = 0
        mask_copy[list(fill_y), list(fill_x)] = 0
        
        # Get the number of nonzero pixels - that is the number of mask pixels that the ellipse does not cover
        full_mask_nonzero = np.count_nonzero(resized_image)
        ellipse_non_zero = np.count_nonzero(mask_copy)

        return ((full_mask_nonzero - ellipse_non_zero) / full_mask_nonzero) * 100
    except:
        print(f'Could not get ellipse for factor size {factor_size}')
        return 0

# Filter Logic

In [10]:
# TODO:
# Read the csv file, which is the output of the extraction module
# Loop over the lines and foreach line extract and read the mask image (using the uuid)
# get the ellipse data from the getMaskBestEllipse function
# set the mask copy values to 0 (black) in the ellipse area
# calculate the number of nonzero pixels in the mask copy
# if it exceeds a predefined value (max_white_pixel_outside_ellipse, need to decide what is the best value) - it's not a good filter
# if it does not exceed - a good filter, insert the line from the input csv to a new output csv

with open(input_csv_file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    input_csv_rows_list = list(csv_reader)
    counter = start_line
    for line in input_csv_rows_list[start_line:end_line + 1]:
        start = time.time()
        counter = counter + 1
        # Get the mask data
        current_mask_uuid = line[uuid_row_index]
        current_mask_filename = current_mask_uuid + images_extension
        current_mask_data = io.imread(masks_images_path + current_mask_filename, as_gray=True)

        factor_size = 2
        ellipse_detected = False
        
        while factor_size <= max_factor_size and not ellipse_detected:
            # Get the coverage percent for the curretn sace factor
            current_cover_percent = getEllipseCoverPercent(current_mask_data, factor_size)

            # If the coverage percentage is greater than the wanted minimum - this is a good mask
            if current_cover_percent > minimum_cover_percent:
                ellipse_detected = True
                output_csv_rows_list.append({
                    "source_file": line[0],
                    "mask_uuid": line[1],
                    "left_x": line[2],
                    "right_x": line[3],
                    "top_y": line[4],
                    "bottom_y": line[5],
                    "id": line[6]
                })
            
            factor_size = factor_size + 1
        end = time.time()
        print(f'line number {counter} - time {end - start} seconds')
            
                    
with open(output_csv_file_path, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=csv_field_names)
    print(f'Writing {len(output_csv_rows_list)} filtered lines to {output_csv_filename}')
    writer.writerows(output_csv_rows_list)

line number 10002 - time 2.5582821369171143 seconds
line number 10003 - time 7.289428949356079 seconds
line number 10004 - time 2.412520170211792 seconds
line number 10005 - time 2.062741994857788 seconds
Could not get ellipse for factor size 4
line number 10006 - time 9.772571325302124 seconds
line number 10007 - time 1.7769036293029785 seconds
line number 10008 - time 0.5091583728790283 seconds
line number 10009 - time 0.37757325172424316 seconds
line number 10010 - time 0.5241715908050537 seconds
line number 10011 - time 12.070249795913696 seconds
line number 10012 - time 1.5917143821716309 seconds
line number 10013 - time 6.302903175354004 seconds
line number 10014 - time 4.000051975250244 seconds
line number 10015 - time 3.9847607612609863 seconds
line number 10016 - time 9.281813621520996 seconds
line number 10017 - time 0.5865800380706787 seconds
line number 10018 - time 3.1948840618133545 seconds
line number 10019 - time 13.099215745925903 seconds
line number 10020 - time 0.487

Could not get ellipse for factor size 2
line number 10157 - time 2.995650053024292 seconds
line number 10158 - time 2.0125060081481934 seconds
line number 10159 - time 0.5644912719726562 seconds
line number 10160 - time 3.130819320678711 seconds
line number 10161 - time 0.48784875869750977 seconds
line number 10162 - time 0.46347761154174805 seconds
line number 10163 - time 3.884873151779175 seconds
line number 10164 - time 4.686697959899902 seconds
line number 10165 - time 2.811781406402588 seconds
line number 10166 - time 0.5704631805419922 seconds
line number 10167 - time 0.4476323127746582 seconds
line number 10168 - time 1.4604432582855225 seconds
line number 10169 - time 2.5800607204437256 seconds
line number 10170 - time 3.050865650177002 seconds
line number 10171 - time 1.767827033996582 seconds
line number 10172 - time 0.5208399295806885 seconds
line number 10173 - time 2.916869878768921 seconds
line number 10174 - time 1.440852165222168 seconds
line number 10175 - time 5.1091

line number 10312 - time 0.4518759250640869 seconds
line number 10313 - time 4.026263236999512 seconds
line number 10314 - time 1.4098906517028809 seconds
line number 10315 - time 3.7108185291290283 seconds
line number 10316 - time 0.5893008708953857 seconds
line number 10317 - time 1.3727693557739258 seconds
line number 10318 - time 2.7221736907958984 seconds
line number 10319 - time 3.55169415473938 seconds
line number 10320 - time 2.570905923843384 seconds
line number 10321 - time 3.621187686920166 seconds
line number 10322 - time 1.6406183242797852 seconds
line number 10323 - time 0.5256390571594238 seconds
line number 10324 - time 4.765605688095093 seconds
line number 10325 - time 4.724816560745239 seconds
line number 10326 - time 4.604082345962524 seconds
Could not get ellipse for factor size 4
line number 10327 - time 7.321387529373169 seconds
line number 10328 - time 2.4316251277923584 seconds
line number 10329 - time 3.0561938285827637 seconds
line number 10330 - time 0.345060

line number 10469 - time 0.4618053436279297 seconds
line number 10470 - time 3.497140645980835 seconds
line number 10471 - time 2.5243844985961914 seconds
line number 10472 - time 5.430614233016968 seconds
line number 10473 - time 3.102206230163574 seconds
line number 10474 - time 7.5179455280303955 seconds
line number 10475 - time 1.5778915882110596 seconds
line number 10476 - time 3.9743998050689697 seconds
line number 10477 - time 2.996018886566162 seconds
line number 10478 - time 4.277891397476196 seconds
line number 10479 - time 3.0642566680908203 seconds
line number 10480 - time 5.8097193241119385 seconds
line number 10481 - time 4.29855489730835 seconds
line number 10482 - time 1.7187328338623047 seconds
line number 10483 - time 0.6264626979827881 seconds
line number 10484 - time 1.8588168621063232 seconds
line number 10485 - time 3.5144338607788086 seconds
line number 10486 - time 2.599989414215088 seconds
line number 10487 - time 0.5588240623474121 seconds
line number 10488 - 

line number 10628 - time 2.8499152660369873 seconds
line number 10629 - time 8.778654336929321 seconds
line number 10630 - time 7.792532205581665 seconds
line number 10631 - time 0.3591799736022949 seconds
line number 10632 - time 2.8830878734588623 seconds
line number 10633 - time 3.5107696056365967 seconds
Could not get ellipse for factor size 3
line number 10634 - time 5.0017991065979 seconds
line number 10635 - time 0.41118764877319336 seconds
line number 10636 - time 0.3905346393585205 seconds
line number 10637 - time 2.155522346496582 seconds
line number 10638 - time 1.709981918334961 seconds
line number 10639 - time 1.954988718032837 seconds
line number 10640 - time 2.3107314109802246 seconds
line number 10641 - time 13.26673674583435 seconds
line number 10642 - time 2.8981382846832275 seconds
line number 10643 - time 3.429347515106201 seconds
line number 10644 - time 1.6405270099639893 seconds
line number 10645 - time 1.7640445232391357 seconds
line number 10646 - time 2.465599

...

line number 14250 - time 2.058471202850342 seconds
line number 14251 - time 3.9291563034057617 seconds
line number 14252 - time 4.692299842834473 seconds
line number 14253 - time 20.21233582496643 seconds
line number 14254 - time 4.524394273757935 seconds
line number 14255 - time 0.5472207069396973 seconds
line number 14256 - time 5.766608715057373 seconds
line number 14257 - time 2.054222345352173 seconds
line number 14258 - time 1.7147486209869385 seconds
line number 14259 - time 0.8242230415344238 seconds
line number 14260 - time 3.047807455062866 seconds
line number 14261 - time 0.6235218048095703 seconds
line number 14262 - time 2.302520751953125 seconds
line number 14263 - time 0.5580463409423828 seconds
line number 14264 - time 3.3590078353881836 seconds
line number 14265 - time 1.6740007400512695 seconds
line number 14266 - time 1.6755385398864746 seconds
line number 14267 - time 0.5880706310272217 seconds
line number 14268 - time 0.5226187705993652 seconds
line number 14269 - 

line number 14408 - time 0.3892369270324707 seconds
line number 14409 - time 3.300410270690918 seconds
line number 14410 - time 0.42793726921081543 seconds
line number 14411 - time 0.6042211055755615 seconds
line number 14412 - time 2.825525999069214 seconds
line number 14413 - time 2.128443479537964 seconds
line number 14414 - time 4.812867164611816 seconds
line number 14415 - time 3.184774875640869 seconds
line number 14416 - time 1.2818057537078857 seconds
line number 14417 - time 2.4467904567718506 seconds
line number 14418 - time 3.983671188354492 seconds
line number 14419 - time 0.41086649894714355 seconds
line number 14420 - time 2.037247657775879 seconds
line number 14421 - time 0.8801395893096924 seconds
line number 14422 - time 1.8157966136932373 seconds
line number 14423 - time 2.5835647583007812 seconds
line number 14424 - time 2.649111032485962 seconds
line number 14425 - time 0.5506813526153564 seconds
line number 14426 - time 0.9522240161895752 seconds
line number 14427 

line number 14566 - time 2.117626428604126 seconds
line number 14567 - time 4.161383867263794 seconds
line number 14568 - time 1.9019217491149902 seconds
line number 14569 - time 1.7417232990264893 seconds
line number 14570 - time 5.011736869812012 seconds
line number 14571 - time 3.77437424659729 seconds
line number 14572 - time 1.881890058517456 seconds
line number 14573 - time 0.6373679637908936 seconds
line number 14574 - time 1.9764904975891113 seconds
line number 14575 - time 7.696151256561279 seconds
line number 14576 - time 4.547419309616089 seconds
line number 14577 - time 3.438049793243408 seconds
line number 14578 - time 0.43494391441345215 seconds
line number 14579 - time 1.9788172245025635 seconds
line number 14580 - time 1.9161009788513184 seconds
line number 14581 - time 2.283721446990967 seconds
line number 14582 - time 1.4715311527252197 seconds
line number 14583 - time 2.6814093589782715 seconds
line number 14584 - time 3.9931836128234863 seconds
line number 14585 - t

line number 14725 - time 2.1737523078918457 seconds
line number 14726 - time 3.662017583847046 seconds
line number 14727 - time 1.8167524337768555 seconds
line number 14728 - time 2.5077590942382812 seconds
line number 14729 - time 0.46731996536254883 seconds
line number 14730 - time 8.194101810455322 seconds
line number 14731 - time 1.616042137145996 seconds
line number 14732 - time 1.6263928413391113 seconds
line number 14733 - time 2.1600241661071777 seconds
line number 14734 - time 0.44188356399536133 seconds
line number 14735 - time 0.4592914581298828 seconds
line number 14736 - time 1.6118683815002441 seconds
line number 14737 - time 1.7996764183044434 seconds
line number 14738 - time 2.674443006515503 seconds
line number 14739 - time 0.39211249351501465 seconds
line number 14740 - time 2.0278093814849854 seconds
line number 14741 - time 0.3434743881225586 seconds
line number 14742 - time 0.4867720603942871 seconds
line number 14743 - time 0.41872501373291016 seconds
line number 

line number 14880 - time 5.702969074249268 seconds
line number 14881 - time 1.7230746746063232 seconds
line number 14882 - time 1.6154875755310059 seconds
line number 14883 - time 4.117214918136597 seconds
line number 14884 - time 1.6147496700286865 seconds
line number 14885 - time 4.156933784484863 seconds
line number 14886 - time 0.7654480934143066 seconds
line number 14887 - time 4.437631845474243 seconds
line number 14888 - time 2.927989959716797 seconds
line number 14889 - time 0.396359920501709 seconds
line number 14890 - time 0.4544944763183594 seconds
line number 14891 - time 0.4906599521636963 seconds
line number 14892 - time 3.458446741104126 seconds
line number 14893 - time 3.9776883125305176 seconds
line number 14894 - time 0.39000868797302246 seconds
line number 14895 - time 5.0544304847717285 seconds
line number 14896 - time 4.484638452529907 seconds
line number 14897 - time 1.9521269798278809 seconds
line number 14898 - time 2.161500930786133 seconds
line number 14899 - 